Load libraries

In [59]:
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)
require(sjstats)
require(broom)

# Load data and set factors

In [110]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)
# mydata$pre <- factor(mydata$pre)
# mydata$main <- factor(mydata$main)

In [111]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163,L,Concentration,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,L,Width,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,C,Area,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10127163,C,Separation,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10232160,L,Concentration,0,0,1,1,1,1,LC,...,1,1,2,1,1,1,1,1,1,4
10232160,L,Width,0,0,0,0,1,1,LC,...,1,1,0,1,1,1,1,1,1,4


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"

# Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)
* student attributes: lab_experience, similar_sim, prior_number_virtual_labs, attitude components

https://www.statmethods.net/stats/anova.html

## Using type 1 and aov package

Without repeated measures but including sid as a term

In [122]:
am1 <- aov(main
    ~  cvs_graph + cvs_table + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + sid, data=mydata)
summary(am1)

                           Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_graph                   1  29.02  29.021 127.387  < 2e-16 ***
cvs_table                   1   0.92   0.922   4.046  0.04491 *  
variable                    3   1.80   0.599   2.630  0.04968 *  
sim_index                   1   2.40   2.405  10.555  0.00125 ** 
pre                         1   7.49   7.485  32.856 1.86e-08 ***
lab_experience              1   0.07   0.065   0.286  0.59338    
similar_sim                 1   0.08   0.078   0.344  0.55768    
prior_number_virtual_labs   1   0.39   0.391   1.717  0.19080    
overall.POCC                1   0.85   0.848   3.723  0.05434 .  
PC1                         1   0.11   0.107   0.470  0.49317    
PC2                         1   0.38   0.378   1.658  0.19862    
sid                       141  98.47   0.698   3.065  < 2e-16 ***
Residuals                 433  98.64   0.228                     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

In [123]:
drop1(am1,~.,test="F")

,Df,Sum of Sq,RSS,AIC,F value,Pr(>F)
<none>,NA,NA,98.64319,-739.7080,NA,NA
cvs_graph,1,2.847763e+00,101.49095,-724.9733,12.500420043,4.507331e-04
cvs_table,1,2.948199e-03,98.64614,-741.6905,0.012941292,9.094809e-01
variable,3,2.053713e+00,100.69690,-733.5918,3.004964024,3.020047e-02
sim_index,1,2.747183e+00,101.39037,-725.5563,12.058916980,5.672906e-04
pre,1,5.046346e-01,99.14782,-738.7076,2.215122837,1.373921e-01
lab_experience,0,1.421085e-14,98.64319,-739.7080,NA,NA
similar_sim,1,1.078127e-03,98.64427,-741.7016,0.004732502,9.451860e-01
prior_number_virtual_labs,0,-2.842171e-14,98.64319,-739.7080,NA,NA
overall.POCC,0,1.421085e-14,98.64319,-739.7080,NA,NA


In [ ]:
test <- Anova(lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + similar_sim + sid, data=mydata),type=3)
test

In [140]:
lm1 <- lm(main
    ~  cvs_table + cvs_graph*pre + variable + sim_index
    + sid, data=mydata)

In [141]:
Anova(lm1)

,Sum Sq,Df,F value,Pr(>F)
cvs_table,8.120167e-04,1,0.00356978,9.523842e-01
cvs_graph,2.846821e+00,1,12.51516916,4.472888e-04
pre,5.063759e-01,1,2.22612479,1.364220e-01
variable,2.244563e+00,3,3.28917482,2.064470e-02
sim_index,2.693305e+00,1,11.84028078,6.358989e-04
sid,1.004755e+02,146,3.02540848,6.635509e-19
cvs_graph:pre,1.498976e-01,1,0.65897860,4.173668e-01
Residuals,9.849437e+01,433,NA,NA


In [142]:
Anova(lm1, type=2)

,Sum Sq,Df,F value,Pr(>F)
cvs_table,8.120167e-04,1,0.00356978,9.523842e-01
cvs_graph,2.846821e+00,1,12.51516916,4.472888e-04
pre,5.063759e-01,1,2.22612479,1.364220e-01
variable,2.244563e+00,3,3.28917482,2.064470e-02
sim_index,2.693305e+00,1,11.84028078,6.358989e-04
sid,1.004755e+02,146,3.02540848,6.635509e-19
cvs_graph:pre,1.498976e-01,1,0.65897860,4.173668e-01
Residuals,9.849437e+01,433,NA,NA


In [143]:
Anova(lm1, type=3)

,Sum Sq,Df,F value,Pr(>F)
(Intercept),8.666325e+00,1,38.09881266,1.547874e-09
cvs_table,8.120167e-04,1,0.00356978,9.523842e-01
cvs_graph,1.946973e+00,1,8.55926366,3.618463e-03
pre,4.178006e-02,1,0.18367311,6.684486e-01
variable,2.244563e+00,3,3.28917482,2.064470e-02
sim_index,2.693305e+00,1,11.84028078,6.358989e-04
sid,1.004755e+02,146,3.02540848,6.635509e-19
cvs_graph:pre,1.498976e-01,1,0.65897860,4.173668e-01
Residuals,9.849437e+01,433,NA,NA


In [120]:
Anova(lm(main
    ~  cvs_table + cvs_graph, data=mydata, contrasts=list(cvs_graph=contr.sum, cvs_table=contr.sum)), type=3)
Anova(lm(main
    ~  cvs_table + cvs_graph, data=mydata), type=3)
# Anova(lm(time ~ topic * sys, data=search, contrasts=list(topic=contr.sum, sys=contr.sum)), type=3)

,Sum Sq,Df,F value,Pr(>F)
(Intercept),875.6046297,1,2431.504843,1.595791e-210
cvs_table,0.9216301,1,2.559315,1.101860e-01
cvs_graph,8.4328385,1,23.417519,1.670149e-06
Residuals,210.6632483,585,NA,NA


,Sum Sq,Df,F value,Pr(>F)
(Intercept),273.7241379,1,760.116546,7.453574e-108
cvs_table,0.9216301,1,2.559315,1.101860e-01
cvs_graph,8.4328385,1,23.417519,1.670149e-06
Residuals,210.6632483,585,NA,NA


In [79]:
# Anova(lm(time ~ topic * sys, data=search, typae=2))
test <- Anova(lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + sid, data=mydata)), type="II")
test

,Sum Sq,Df,F value,Pr(>F)
cvs_table,0.002948199,1,0.012941292,9.094809e-01
cvs_graph,2.847762816,1,12.500420043,4.507331e-04
variable,2.053712943,3,3.004964024,3.020047e-02
sim_index,2.747182515,1,12.058916980,5.672906e-04
pre,0.504634598,1,2.215122837,1.373921e-01
lab_experience,0.036568667,1,0.160520286,6.888756e-01
similar_sim,0.001078127,1,0.004732502,9.451860e-01
prior_number_virtual_labs,0.019467908,1,0.085455511,7.701761e-01
overall.POCC,0.229393925,1,1.006937936,3.161980e-01
PC1,0.020122602,1,0.088329328,7.664544e-01


In [82]:
# Anova(lm(time ~ topic * sys, data=search, type=2))
test <- Anova(lm(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + sid, data=mydata))
test

,Sum Sq,Df,F value,Pr(>F)
cvs_table,0.001449097,1,0.01067749,9.177474e-01
cvs_graph,0.303598914,1,2.23703084,1.354680e-01
variable,1.027500181,3,2.52366909,5.719406e-02
sim_index,0.349621166,1,2.57614007,1.092150e-01
pre,0.645921480,1,4.75939208,2.967593e-02
lab_experience,0.015486235,1,0.11410840,7.356781e-01
similar_sim,0.856103746,1,6.30809396,1.238176e-02
prior_number_virtual_labs,0.549335309,1,4.04770890,4.485027e-02
overall.POCC,0.583487826,1,4.29935747,3.871750e-02
PC1,1.543510280,1,11.37316350,8.120718e-04


In [83]:
# Anova(lm(time ~ topic * sys, data=search, type=2))
test <- Anova(lm(quant_score
    ~  cvs_graph +cvs_table + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + sid, data=mydata))
test

,Sum Sq,Df,F value,Pr(>F)
cvs_graph,0.303598914,1,2.23703084,1.354680e-01
cvs_table,0.001449097,1,0.01067749,9.177474e-01
variable,1.027500181,3,2.52366909,5.719406e-02
sim_index,0.349621166,1,2.57614007,1.092150e-01
pre,0.645921480,1,4.75939208,2.967593e-02
lab_experience,0.015486235,1,0.11410840,7.356781e-01
similar_sim,0.856103746,1,6.30809396,1.238176e-02
prior_number_virtual_labs,0.549335309,1,4.04770890,4.485027e-02
overall.POCC,0.583487826,1,4.29935747,3.871750e-02
PC1,1.543510280,1,11.37316350,8.120718e-04


In [86]:
test <- Anova(lm(main
     ~ pre*variable, data=mydata))
test

,Sum Sq,Df,F value,Pr(>F)
pre,11.54529050,1,29.57224896,7.949694e-08
variable,3.09637285,3,2.64369582,4.846810e-02
pre:variable,0.09876208,3,0.08432347,9.685876e-01
Residuals,226.43758007,580,NA,NA


In [87]:
test <- Anova(lm(main
     ~ pre*variable, data=mydata), type=2)
test

,Sum Sq,Df,F value,Pr(>F)
pre,11.54529050,1,29.57224896,7.949694e-08
variable,3.09637285,3,2.64369582,4.846810e-02
pre:variable,0.09876208,3,0.08432347,9.685876e-01
Residuals,226.43758007,580,NA,NA


In [ ]:
am1 <- aov(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am1)
# stats = tidy(am1)
# ss_error = filter(stats,term=="Residuals")$sumsq
# stats['eta_sq'] <- stats$sumsq/(ss_error+stats$sumsq)
# stats

We see that the variance from having repeated measures for each student (first table) has been removed from the overall model error and thus separated in the output.

Looking the second table we notice that both cvs are significant, cvs_graph more significant than cvs_table, variable is significant and so is sim_index.

eta squares, and post hoc test are coming!

In [ ]:
summary(am1[["Error: sid:variable"]])

## Using type 3 and anova package

We create a dataframe to tell the anova how "variable" leades to multiple between subject measures.

For info on why Type 1,2,3: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/

In [ ]:
variables <- factor(c("Width","Area","Concentration","Separation"))
measures <- factor(rep(c("pre", "main", "cvs_table","cvs_graph","sim_index","similar_sim"), c(4,4,4,4,4,4)),
levels=c("pre", "main", "cvs_table","cvs_graph","sim_index","similar_sim"))
vars <- ordered(rep(variables, 6))
idata <- data.frame(measures,vars)
idata

Try repeating this blog: http://psych.wisc.edu/moore/Rpdf/610-R8_OneWayWithin.pdf

In [ ]:
mod.ok <- lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre + sid
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
av.ok <- Anova(mod.ok, type=2)#, idata=idata, idesign=~pre*variable*main*cvs_table*cvs_graph*sim_index*similar_sim)
av.ok

In [ ]:
 Anova(mod.ok, type=2)

In [ ]:
 Anova(mod.ok)

In [ ]:
mod.ok <- lm(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
variables <- factor(cbind("Width","Area","Concentration","Separation"))
idata <- data.frame(variables)
av.ok <- Anova(mod.ok, idata=idata, idesign=~variable,type=3)
av.ok

## Redo models per variable (so without repeated measures)

https://datascienceplus.com/two-way-anova-with-repeated-measures/

In [ ]:
for (var in c("Width","Area","Concentration","Separation")){
    print ("")
    print(rep(var,9))
    var_data = filter(mydata,variable==var)
    mod.ok <- lm(main
    ~  cvs_table + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=var_data)
    av.three <- Anova(mod.ok, type=3)
#     av.two <- Anova(mod.ok, type=2)
#     av.one <- Anova(mod.ok)
    print(av.three)
#     print(av.two)
#     print(av.one)
}

In [ ]:
# am1_graph <- aov(main
#     ~ cvs_graph + variable + sim_index + pre
#     + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
#     +  PC1 + PC2 + Error(sid/variable), data=filter(mydata, cvs_table == 1))
# summary(am1_graph)

# Stat model 2: Predicting transfer data

In [ ]:
am2 <- aov(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am2)

As expected, CVS doesn't predict quant transfer scores.

Post hoc by variable and similar sim (weither students had worked with a similar sim before) coming soon.

## Using type 3 and anova package

In [ ]:
mod.ok <- lm(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, data=mydata)
variables <- factor(cbind("Width","Area","Concentration","Separation"))
idata <- data.frame(variables)
av.ok <- Anova(mod.ok, idata=idata, idesign=~variables,type=3)
av.ok

# Stat model 3: Predicting the use of CVS

## For cvs_table

In [ ]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

## For cvs_graph

In [ ]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

# Example from R package "car"
See page 9 of https://cran.r-project.org/web/packages/car/car.pdf

In [ ]:
?OBrienKaiser

In [ ]:
OBrienKaiser

In [ ]:
phase <- factor(rep(c("pretest", "posttest", "followup"), c(5, 5, 5)),
levels=c("pretest", "posttest", "followup"))
hour <- ordered(rep(1:5, 3))
idata <- data.frame(phase, hour)
idata

In [ ]:
mod.ok <- lm(cbind(pre.1, pre.2, pre.3, pre.4, pre.5,
post.1, post.2, post.3, post.4, post.5,
fup.1, fup.2, fup.3, fup.4, fup.5) ~ treatment*gender,
data=OBrienKaiser)
(av.ok <- Anova(mod.ok, idata=idata, idesign=~phase*hour,type=3))
summary(av.ok, multivariate=FALSE)